In [4]:
import cv2 as cv
import numpy as np
import mysql.connector
import datetime



#Initialise Camera
vid = cv.VideoCapture(0)
dt=datetime.date.today()
# connecting to the db
mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="cisco_thingqbator"
    )
db_cursor1=mydb.cursor()

#Create a cascade classifier object
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
faces=[]

#No. of frames
nof = 0

#List to store each face, to be done later
face_list=[]
path = "imgs/"

while True:
    #checks if the day hasn't changed or if the month has changed
    now=datetime.date.today()
    if dt.day+1==now.day or dt.day>now.day:
        dt=datetime.date.today()
        face_list=[]
#       add a query to send number of faces to the db
#         HERE

    #check returns bool, frame returns the image
    check, frame = vid.read()
    if not check:
        continue
        
    #Increment number of frames
    nof += 1
    
    #Convert image to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    #Detect multiple faces. Returns a list of faces in the current frame.
    faces = face_cascade.detectMultiScale(gray, 1.5, 5)
    
    if len(faces)==0:
        continue
    
    #Sort the faces in decreasing order on basis of features
    faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
    
    #Draw a rectangle around each face
    for top, left, bottom, right in faces:
        gray = cv.rectangle(gray, (top, left), (top+bottom, left+right), (0,255,0), 1)
        
    
    #Process only if faces is not empty
    if len(faces)!=0:
        for face in faces[:1]:
            x, y, w, h = face
            
            # Get the face ROI(region of interest)
            offset = 7
            face_section = gray[y-offset:y+h+offset, x-offset:x+w+offset]
            face_section = cv.resize(face_section, (100, 100))
                                
            #Append the ROI in the list after every 10 frames
#             enc_face_section = fr.api.face_encodings(face_section, None, "small")
#             print(type(enc_face_section))
            if nof % 10 == 0 :
                face_list.append(face_section)
        
        
    #Show the faces with rectangle drawn on them
    cv.imshow("Face", gray)
    
    #Wait for 1ms per frame. Quit if 'q' pressed.
    key = cv.waitKey(1)
    if key == ord('q'):
        break
    
#Print no. of frames
print(nof)
for i in face_list:
    print(i)
face_list = np.asarray(face_list)

for i in range(len(face_list)):
    np.save(path + str(i), face_list[i])
vid.release()
cv.destroyAllWindows()


462
[[ 42  45  54 ... 192 191 192]
 [ 69  69  76 ... 192 192 192]
 [ 76  78  82 ... 192 192 191]
 ...
 [ 10   9   9 ...  31  32  38]
 [ 10  10  10 ...  26  28  33]
 [ 11  10  10 ...  26  26  29]]
[[ 95  95  96 ... 185 193 194]
 [ 94  94  95 ... 182 192 194]
 [ 94  93  92 ... 181 192 193]
 ...
 [ 12  10  10 ...  57  55  65]
 [ 11  10  10 ...  53  58  61]
 [ 11  10  10 ...  47  56  54]]
[[105 105 106 ... 108 117 153]
 [105 104 103 ... 107 126 166]
 [103 102 102 ... 110 144 169]
 ...
 [ 12  12  13 ...  82  90  95]
 [ 12  12  13 ...  80  89  92]
 [ 12  13  12 ...  79  89  96]]
[[106 105 105 ... 194 193 193]
 [106 104 105 ... 194 194 194]
 [104 103 104 ... 194 195 195]
 ...
 [ 14  14  14 ... 142 150 150]
 [ 13  13  13 ... 137 149 151]
 [ 15  14  14 ... 138 146 148]]
[[187 185 181 ... 191 190 191]
 [188 185 179 ... 192 191 190]
 [188 185 180 ... 192 192 191]
 ...
 [151 150 148 ...  21  21  23]
 [150 150 150 ...  23  22  23]
 [149 152 156 ...  25  24  25]]
[[149 189 191 ... 194 192 192]
 [159

In [10]:
import cv2

# Initialize camera
cap = cv2.VideoCapture(0) # Capturing video

# Load the haar cascade for frontal face
face_cascade = cv2.CascadeClassifier('C:/Users/Ayush/Open CV/haarcascade_frontalface_default.xml')

skip = 0 #frame number count print
face_data = [] 
dataset_path = 'C:/Users/Ayush/Desktop/imgs/'

file_name = input("Enter the name of the person: ")

while True:
    ret, frame = cap.read() #read frame ret = return value
    if ret == False:
        continue
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # grayscale convert

    # Detect multi faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) #Capture multiple faces
    k = 1

    faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True) #faces coming sort them on basis of features 

    # update the frame number
    skip += 1

    for face in faces[:1]:
        x, y, w, h = face

        # Get the face ROI
        offset = 7
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        if skip % 10 == 0:
            face_data.append(face_section)
            print (len(face_data))

        # Display the face ROI
        cv2.imshow(str(k), face_section)
        k += 1

        # Draw rectangle in the original image
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    cv2.imshow("Faces", frame)

    if cv2.waitKey(1) & 0xFF == ord('n'):
        break

# Convert face list to numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
print (face_data.shape)

# Save the dataset in filesystem
np.save(dataset_path + file_name, face_data)
print ("Dataset saved at: {}".format(dataset_path + file_name + '.npy'))

cv2.destroyAllWindows()

Enter the name of the person: Astha
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
import numpy as np
import cv2
import os
def distance(v1, v2):
    # Eucledian distance
    return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
    dist = []

    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]
################################

# Initialize camera
cap = cv2.VideoCapture(0)

# Load the haar cascade for frontal face
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface.xml')

dataset_path = '/Users/kavita/Desktop/'

face_data = []
labels = []
class_id = 0

# Dataset prepration
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)

        target = class_id * np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1, 1))
print (face_labels.shape)
print (face_dataset.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print (trainset.shape)

names= []
# r=root, d=directories, f = files
for r, d, f in os.walk(dataset_path):
    for file in f:
        if '.npy' in file:
            names.append(os.path.splitext(file)[0])

for f in names:
    print(f)



font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    ret, frame = cap.read()
    if ret == False:
        continue
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect multi faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for face in faces:
        x, y, w, h = face

        # Get the face ROI
        offset = 7
        face_section = frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100, 100))

        out = knn(trainset, face_section.flatten())

        # Draw rectangle in the original image
        #cv2.putText(frame, names[int(out)],(x,y-10), font, 1,(255,0,0),2,cv2.LINE_AA)
        #Distinct Faces algo
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)

    cv2.imshow("Faces", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()